In [ ]:
!pip install astroquery
!pip install astropy

# 1. Gaia DR3 Catalog


### 1.1 Querying data with the apparent magnitude threshold

In [ ]:
from astroquery.gaia import Gaia
from astropy.coordinates import SkyCoord
import astropy.units as u
import pandas as pd
import numpy as np
import time
import os

# 외계행성 위치 데이터 정의
exoplanet_coord_Gliese = SkyCoord(ra='15h19m26.82694s', dec='-0d43m20.1895s', distance=6.3*u.pc, frame='icrs')
exoplanet_coord_K2 = SkyCoord(ra='11h30m14.51774s', dec='+07d35m18.2553s', distance=38*u.pc, frame='icrs')

# 로그인 (필요한 경우)
# Gaia.login(user='ylee08', password='Jobcho!127!')  # 보안상 로그인 정보는 코드에 포함하지 않는 것이 좋습니다.

# 데이터 처리 및 CSV 파일 저장 함수
def process_and_save_data(data, exoplanet_coord, csv_filename):
    # 파랄락스를 이용해 거리 계산 (파랄락스는 밀리초단위로 제공됨)
    # 거리 (pc) = 1000 / 파랄락스 (mas)
    # 파랄락스의 상대 불확실성이 너무 큰 경우 필터링
    parallax = data['parallax']
    parallax_error = data['parallax_error']
    relative_error = parallax_error / parallax

    # 유효한 거리 계산을 위해 파랄락스 > 0 및 상대 불확실성 < 20%인 경우만 사용
    valid = (parallax > 0) & (relative_error < 0.2)
    data = data[valid].copy()

    # 거리 계산
    data['distance'] = 1000.0 / parallax  # 거리 (pc)

    # 별들의 좌표 생성
    star_coords = SkyCoord(ra=data['ra'].values * u.degree,
                           dec=data['dec'].values * u.degree,
                           distance=data['distance'].values * u.pc,
                           frame='icrs')

    # 외계행성 기준으로 별들의 상대 위치 계산
    relative_coords = star_coords.cartesian - exoplanet_coord.cartesian

    # 외계행성 기준에서 별들까지의 새로운 거리 계산
    new_distances = relative_coords.norm()

    # 원래 절대 등급 계산
    data['abs_mag'] = data['phot_g_mean_mag'] - 5 * np.log10(data['distance']) + 5

    # 새로운 겉보기 등급 계산
    new_app_mag = data['abs_mag'] + 5 * np.log10(new_distances.value) - 5

    # 새로운 겉보기 등급을 데이터에 추가
    data['new_phot_g_mean_mag'] = new_app_mag

    # 조건을 만족하는 데이터 필터링
    threshold_mag = 18
    filtered_data = data[new_app_mag < threshold_mag]

    if not filtered_data.empty:
        # 폴더 경로 추출 및 생성
        directory = os.path.dirname(csv_filename)
        if directory and not os.path.exists(directory):
            os.makedirs(directory)

        # CSV 파일에 데이터 추가 (파일이 없으면 생성)
        file_exists = os.path.exists(csv_filename)
        filtered_data.to_csv(csv_filename, mode='a', index=False, header=not file_exists)

# RA와 Dec의 시작 각도와 끝 각도, 분할 개수 설정
ra_start_total = 0      # RA 시작 각도 (0° 이상)
ra_end_total = 360      # RA 끝 각도 (360° 이하)
num_ra_partitions = 2 # RA 분할 개수

dec_start_total = -90    # Dec 시작 각도 (-90° 이상)
dec_end_total = 90       # Dec 끝 각도 (+90° 이하)
num_dec_partitions = 2  # Dec 분할 개수

# 분할 간격 계산
ra_step = (ra_end_total - ra_start_total) / num_ra_partitions
dec_step = (dec_end_total - dec_start_total) / num_dec_partitions

csv_filename_Gliese = 'filtered_gaia_data_Gliese.csv'
csv_filename_K2 = 'filtered_gaia_data_K2.csv'

for ra_index in range(num_ra_partitions):
    ra_start = ra_start_total + ra_index * ra_step
    ra_end = ra_start + ra_step

    for dec_index in range(num_dec_partitions):
        dec_start = dec_start_total + dec_index * dec_step
        dec_end = dec_start + dec_step

        query = f"""
        SELECT
          gs.source_id,
          gs.ra,
          gs.dec,
          gs.phot_g_mean_mag,
          gs.parallax,
          gs.parallax_error,
          ap.teff_gspphot,
          ap.radius_gspphot
        FROM
          gaiadr3.gaia_source AS gs
        LEFT JOIN
          gaiadr3.astrophysical_parameters AS ap
        ON
          gs.source_id = ap.source_id
        WHERE
          gs.ra >= {ra_start} AND gs.ra < {ra_end}
          AND gs.dec >= {dec_start} AND gs.dec < {dec_end}
          AND gs.phot_g_mean_mag <= 8
          AND (ap.teff_gspphot IS NOT NULL OR ap.teff_gspphot IS NULL)  -- teff와 radius는 필수 조건에서 제외
          AND gs.parallax IS NOT NULL
          AND gs.parallax_error IS NOT NULL
        """

        try:
            # 비동기 쿼리 실행
            job = Gaia.launch_job_async(query, dump_to_file=False)
            results = job.get_results()

            # 결과가 있으면 데이터 처리 및 저장
            if len(results) == 0:
                continue

            data = results.to_pandas()

            # 데이터 처리 및 CSV 파일 저장 함수 호출
            process_and_save_data(data, exoplanet_coord_Gliese, csv_filename_Gliese)
            process_and_save_data(data, exoplanet_coord_K2, csv_filename_K2)

            print(f'RA [{ra_start}, {ra_end}], Dec [{dec_start}, {dec_end}] 영역 처리 완료.')

            # 필요한 경우 대기 시간 추가
            time.sleep(1)

        except Exception as e:
            print(f'쿼리 실패: RA [{ra_start}, {ra_end}], Dec [{dec_start}, {dec_end}]')
            print(f'에러 메시지: {e}')
            continue


INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
RA [0.0, 180.0], Dec [-90.0, 0.0] 영역 처리 완료.


INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
RA [0.0, 180.0], Dec [0.0, 90.0] 영역 처리 완료.


INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
RA [180.0, 360.0], Dec [-90.0, 0.0] 영역 처리 완료.


INFO:astroquery:Query finished.


INFO: Query finished. [astroquery.utils.tap.core]
RA [180.0, 360.0], Dec [0.0, 90.0] 영역 처리 완료.


### 1.2 New apparent magnitude for each exoplanets

In [ ]:
exoplanet_coord = SkyCoord(ra='15h19m26.82694s', dec='-0d43m20.1895s', distance=6.3*u.pc, frame='icrs')
# exoplanet_coord = SkyCoord(ra='11h30m14.51774s', dec='+07d35m18.2553s', distance=38*u.pc, frame='icrs')

In [ ]:
data = pd.read_csv('Gaia_mag16_data.csv')

parallax = data['parallax']
parallax_error = data['parallax_error']
relative_error = parallax_error / parallax

# 유효한 거리 계산을 위해 파랄락스 > 0 및 상대 불확실성 < 20%인 경우만 사용
valid = (parallax > 0) & (relative_error < 0.2)
data = data[valid].copy()

# 거리 계산
data['distance'] = 1000.0 / parallax  # 거리 (pc)

# 별들의 좌표 생성
star_coords = SkyCoord(ra=data['ra'].values * u.degree,
                        dec=data['dec'].values * u.degree,
                        distance=data['distance'].values * u.pc,
                        frame='icrs')

# 외계행성 기준으로 별들의 상대 위치 계산
relative_coords = star_coords.cartesian - exoplanet_coord.cartesian

# 외계행성 기준에서 별들까지의 새로운 거리 계산
new_distances = relative_coords.norm()

# 원래 절대 등급 계산
data['abs_mag'] = data['phot_g_mean_mag'] - 5 * np.log10(data['distance']) + 5

# 새로운 겉보기 등급 계산
new_app_mag = data['abs_mag'] + 5 * np.log10(new_distances.value) - 5

# 새로운 겉보기 등급을 데이터에 추가
data['new_phot_g_mean_mag'] = new_app_mag



### 1.3 Frame Transformation

In [ ]:
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy.coordinates import CartesianRepresentation, UnitSphericalRepresentation
import astropy.units as u
import os

# --- 함수 정의 ---

def transform_coordinates(star_coords, exoplanet_coord):
    """
    외계행성 기준으로 별들의 상대적인 좌표와 거리를 계산합니다.
    """
    # 외계행성의 위치를 Cartesian 좌표로 변환
    exoplanet_cartesian = exoplanet_coord.cartesian

    # 별들의 위치를 Cartesian 좌표로 변환
    star_cartesian = star_coords.cartesian

    # 외계행성에서 본 별들의 위치 계산 (벡터 차이)
    relative_cartesian = star_cartesian - exoplanet_cartesian

    # 상대 위치를 새로운 SkyCoord 객체로 생성 (Cartesian 표현)
    relative_coords_cartesian = SkyCoord(
        x=relative_cartesian.x,
        y=relative_cartesian.y,
        z=relative_cartesian.z,
        representation_type='cartesian',
        frame='icrs'
    )

    # 새로운 거리 계산 (외계행성 기준)
    new_distances = relative_cartesian.norm()

    return relative_coords_cartesian, new_distances

def calculate_normalized_radius(apparent_magnitude, min_radius=0.0005, max_radius=0.01):
    """
    별의 겉보기 등급을 기반으로 별의 크기를 정규화합니다.
    """
    # 겉보기 등급이 NaN인 경우 radius도 NaN으로 설정
    with np.errstate(invalid='ignore'):
        radius = min_radius + (max_radius - min_radius) * 10 ** (-0.2 * apparent_magnitude)
    return radius

def process_exoplanet_data(csv_filename, exoplanet_coord, exoplanet_name, magnitude_limits, observer_location, observation_time, output_formats=['json']):
    """
    주어진 외계행성에 대해 데이터를 처리하고, 지정된 겉보기 등급 기준으로 필터링된 JSON 파일을 생성합니다.

    Parameters:
        csv_filename (str): 입력 CSV 파일 경로
        exoplanet_coord (SkyCoord): 외계행성의 SkyCoord 객체
        exoplanet_name (str): 외계행성의 이름 (파일명에 사용)
        magnitude_limits (list): 겉보기 등급 필터링 기준 리스트
        observer_location (EarthLocation): 관측자의 지리적 위치
        observation_time (Time): 관측 시간
        output_formats (list): 저장할 파일 형식 리스트 (기본값: ['json'])
    """
    # CSV 파일 읽기
    df = pd.read_csv(csv_filename)

    # 필수 필드가 있는지 확인하고 없으면 제외
    required_fields = ['ra', 'dec', 'parallax']
    df = df.dropna(subset=required_fields)

    # 거리 계산 (parallax는 밀리초 단위)
    df['distance'] = 1000.0 / df['parallax']

    # 거리의 유효성 검사 (예: 음수 거리 제외)
    df = df[df['distance'] > 0]

    # 별들의 SkyCoord 객체 생성
    star_coords = SkyCoord(
        ra=df['ra'].values * u.degree,
        dec=df['dec'].values * u.degree,
        distance=df['distance'].values * u.pc,
        frame='icrs'
    )

    # 별들의 SkyCoord 객체 생성 (거리 없이)
    star_coords_no_distance = SkyCoord(
        ra=df['ra'].values * u.degree,
        dec=df['dec'].values * u.degree,
        frame='icrs'
    )

    # AltAz 프레임 생성
    altaz_frame = AltAz(obstime=observation_time, location=observer_location)

    # 거리 정보가 있는 별들만 AltAz 변환 수행
    with np.errstate(invalid='ignore'):
        star_altaz = star_coords_no_distance.transform_to(altaz_frame)

    # AltAz 좌표에서 ENU 좌표계로 변환 (단위 벡터)
    altaz_cartesian = star_altaz.represent_as(UnitSphericalRepresentation).represent_as(CartesianRepresentation)

    # 외계행성 기준으로 좌표 변환
    relative_coords, new_distances = transform_coordinates(star_coords, exoplanet_coord)

    # ENU 좌표 추출
    x_enu = relative_coords.x.value
    y_enu = relative_coords.y.value
    z_enu = relative_coords.z.value

    # 정규화된 반지름 계산 (NaN 처리 포함)
    normalized_radius = calculate_normalized_radius(df['new_phot_g_mean_mag'].values)

    # DataFrame 업데이트
    df_processed = pd.DataFrame({
        'source_id': df.get('source_id') if 'source_id' in df.columns else df.get('SOURCE_ID'),  # 대소문자 처리
        'absolute_magnitude': df.get('abs_mag'),
        'apparent_magnitude': df.get('new_phot_g_mean_mag'),
        'radius': df.get('radius_gspphot'),
        'normalized_radius': normalized_radius,
        'temperature_celsius': df.get('teff_gspphot') - 273.15 if 'teff_gspphot' in df.columns else np.nan,  # 켈빈을 섭씨로 변환, 필드 없으면 NaN
        'distance_pc': new_distances.value,  # 외계행성 기준 거리 (parsec 단위)
        'x_enu': x_enu,
        'y_enu': y_enu,
        'z_enu': z_enu
    })

    # ENU 벡터의 노름 계산 (NaN 처리 포함)
    df_processed['enu_norm'] = np.sqrt(df_processed['x_enu']**2 + df_processed['y_enu']**2 + df_processed['z_enu']**2)

    # 노름이 0인 경우를 피하기 위해 작은 값을 더함 (필요시)
    epsilon = 1e-10
    df_processed['enu_norm'] = df_processed['enu_norm'].replace(0, epsilon)

    # 정규화된 ENU 좌표 계산 (단위 벡터)
    df_processed['x_normalized'] = df_processed['x_enu'] / df_processed['enu_norm']
    df_processed['y_normalized'] = df_processed['y_enu'] / df_processed['enu_norm']
    df_processed['z_normalized'] = df_processed['z_enu'] / df_processed['enu_norm']

    # 원래 ENU 좌표와 노름 열 삭제
    df_processed.drop(['x_enu', 'y_enu', 'z_enu', 'enu_norm'], axis=1, inplace=True)

    # 기타 필드에서 NaN 값 처리 (필요시)
    # 예: 절대 등급이 NaN인 경우 계산 불가, 같은 방식으로 처리할 수 있음
    df_processed['absolute_magnitude'] = df_processed['absolute_magnitude'].replace([np.inf, -np.inf], np.nan)
    df_processed['temperature_celsius'] = df_processed['temperature_celsius'].replace([np.inf, -np.inf], np.nan)

    # 필터링 및 파일 저장
    for mag in magnitude_limits:
        # 필터링: apparent_magnitude <= mag
        df_filtered = df_processed[df_processed['apparent_magnitude'] <= mag].copy()

        # NaN을 포함한 행을 제외하거나 유지 (원하는 대로 선택)
        # 예: NaN을 포함한 행도 저장하려면 필터링 조건에 추가하지 않음
        # 필요에 따라 특정 필드를 기준으로 NaN을 제외할 수 있음

        # 파일 이름 설정
        base_filename = f'{exoplanet_name}_mag{mag}'

        if 'json' in output_formats:
            output_json = f'{base_filename}.json'
            df_filtered.to_json(output_json, orient='records', lines=False, indent=4, default_handler=str)
            print(f"Saved {len(df_filtered)} stars with apparent magnitude <= {mag} to '{output_json}'")

    print(f"Processing for {exoplanet_name} completed.\n")

# --- 메인 스크립트 ---

def main():
    # 겉보기 등급 기준 설정
    magnitude_limits = [4, 6, 8]

    # 관측자 위치 설정 (위도, 경도)
    # 37° 23′ 41.01″ N, 127° 6′ 40.35″ E
    lat_deg = 37 + 23/60 + 41.01/3600  # 37.394725°
    lon_deg = 127 + 6/60 + 40.35/3600  # 127.1112083°

    observer_location = EarthLocation(lat=lat_deg * u.deg, lon=lon_deg * u.deg, height=0 * u.m)

    # 관측 시간 설정 (현재 시간)
    observation_time = Time.now()

    # 외계행성의 위치 정의
    exoplanets = {
        'K2-18': SkyCoord(
            ra='11h30m14.51774s',
            dec='+07d35m18.2553s',
            distance=38.10 * u.pc,
            frame='icrs'
        ),
        'Gliese_581': SkyCoord(
            ra='15h19m26.8250s',
            dec='-07d43m20.209s',
            distance=6.279 * u.pc,
            frame='icrs'
        )
    }

    # 각 외계행성에 대해 데이터 처리
    for exoplanet_name, exoplanet_coord in exoplanets.items():
        if exoplanet_name == 'K2-18':
            csv_filename = 'filtered_gaia_data_K2.csv'
        elif exoplanet_name == 'Gliese_581':
            csv_filename = 'filtered_gaia_data_Gliese.csv'
        else:
            print(f"Unknown exoplanet: {exoplanet_name}")
            continue

        # 데이터 처리 함수 호출
        process_exoplanet_data(
            csv_filename=csv_filename,
            exoplanet_coord=exoplanet_coord,
            exoplanet_name=exoplanet_name,
            magnitude_limits=magnitude_limits,
            observer_location=observer_location,
            observation_time=observation_time,
            output_formats=['json']  # JSON만 저장
        )

if __name__ == "__main__":
    main()


Saved 499 stars with apparent magnitude <= 4 to 'K2-18_mag4.json'
Saved 6369 stars with apparent magnitude <= 6 to 'K2-18_mag6.json'
Saved 55574 stars with apparent magnitude <= 8 to 'K2-18_mag8.json'
Processing for K2-18 completed.

Saved 541 stars with apparent magnitude <= 4 to 'Gliese_581_mag4.json'
Saved 6469 stars with apparent magnitude <= 6 to 'Gliese_581_mag6.json'
Saved 60641 stars with apparent magnitude <= 8 to 'Gliese_581_mag8.json'
Processing for Gliese_581 completed.



# 2. Hipparcos Catalog

### 2.1 Load the Hipparcos dataset

In [ ]:
from astroquery.vizier import Vizier
from astropy.coordinates import SkyCoord
import astropy.units as u
import pandas as pd
import numpy as np
import time
import os

# VizieR 설정: Hipparcos 메인 카탈로그 "I/239/hip_main"
Vizier.ROW_LIMIT = -1  # 모든 행을 가져옵니다.

# Hipparcos 카탈로그 ID
hipparcos_catalog = "I/239/hip_main"

# CSV 파일 저장 경로
hip_csv_filename = 'hipparcos_data.csv'

# 이미 CSV 파일이 존재하면 로드, 아니면 다운로드
if os.path.exists(hip_csv_filename):
    hip_data = pd.read_csv(hip_csv_filename)
    print(f"기존의 Hipparcos 데이터 '{hip_csv_filename}'를 로드했습니다.")
else:
    print("VizieR에서 Hipparcos 데이터를 다운로드 중입니다...")
    try:
        result = Vizier.get_catalogs(hipparcos_catalog)
        hip_table = result[0]
        hip_data = hip_table.to_pandas()
        hip_data.to_csv(hip_csv_filename, index=False)
        print(f"Hipparcos 데이터를 '{hip_csv_filename}'에 저장했습니다.")
    except Exception as e:
        print("Hipparcos 데이터 다운로드에 실패했습니다.")
        print(f"에러 메시지: {e}")
        hip_data = pd.DataFrame()


기존의 Hipparcos 데이터 'hipparcos_data.csv'를 로드했습니다.


### 2.2 Frame transformation & new apparent magnitude calculation

In [ ]:
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord, EarthLocation, AltAz
from astropy.time import Time
from astropy.coordinates import CartesianRepresentation, UnitSphericalRepresentation
import astropy.units as u
import os

# --- 함수 정의 ---

def transform_coordinates(star_coords, exoplanet_coord):
    """
    외계행성 기준으로 별들의 상대적인 좌표와 거리를 계산합니다.
    """
    # 외계행성의 위치를 Cartesian 좌표로 변환
    exoplanet_cartesian = exoplanet_coord.cartesian

    # 별들의 위치를 Cartesian 좌표로 변환
    star_cartesian = star_coords.cartesian

    # 외계행성에서 본 별들의 위치 계산 (벡터 차이)
    relative_cartesian = star_cartesian - exoplanet_cartesian

    # 상대 위치를 새로운 SkyCoord 객체로 생성 (Cartesian 표현)
    relative_coords_cartesian = SkyCoord(
        x=relative_cartesian.x,
        y=relative_cartesian.y,
        z=relative_cartesian.z,
        representation_type='cartesian',
        frame='icrs'
    )

    # 새로운 거리 계산 (외계행성 기준)
    new_distances = relative_cartesian.norm()

    return relative_coords_cartesian, new_distances

def process_exoplanet_data(csv_filename, exoplanet_coord, exoplanet_name, magnitude_limits, observer_location, observation_time, output_formats=['json']):
    """
    주어진 외계행성에 대해 데이터를 처리하고, 지정된 겉보기 등급 기준으로 필터링된 JSON 파일을 생성합니다.

    Parameters:
        csv_filename (str): 입력 CSV 파일 경로
        exoplanet_coord (SkyCoord): 외계행성의 SkyCoord 객체
        exoplanet_name (str): 외계행성의 이름 (파일명에 사용)
        magnitude_limits (list): 겉보기 등급 필터링 기준 리스트
        observer_location (EarthLocation): 관측자의 지리적 위치
        observation_time (Time): 관측 시간
        output_formats (list): 저장할 파일 형식 리스트 (기본값: ['json'])
    """
    # CSV 파일 읽기
    data = pd.read_csv('hipparcos_data.csv')
    df = data.copy()

    # 필수 필드가 있는지 확인하고 없으면 제외
    required_fields = ['RAICRS', 'DEICRS', 'Plx']
    df = df.dropna(subset=required_fields)

    # 거리 계산 (파랄락스는 밀리초 단위)
    df['distance'] = 1000.0 / df['Plx']

    # 거리의 유효성 검사 (예: 음수 거리 제외)
    df = df[df['distance'] > 0]

    # 별들의 SkyCoord 객체 생성
    star_coords = SkyCoord(
        ra=df['RAICRS'].values * u.degree,
        dec=df['DEICRS'].values * u.degree,
        distance=df['distance'].values * u.pc,
        frame='icrs'
    )

    # 별들의 SkyCoord 객체 생성 (거리 없이)
    star_coords_no_distance = SkyCoord(
        ra=df['RAICRS'].values * u.degree,
        dec=df['DEICRS'].values * u.degree,
        frame='icrs'
    )

    # AltAz 프레임 생성
    altaz_frame = AltAz(obstime=observation_time, location=observer_location)

    # 거리 정보가 있는 별들만 AltAz 변환 수행
    with np.errstate(invalid='ignore'):
        star_altaz = star_coords_no_distance.transform_to(altaz_frame)

    # AltAz 좌표에서 ENU 좌표계로 변환 (단위 벡터)
    altaz_cartesian = star_altaz.represent_as(UnitSphericalRepresentation).represent_as(CartesianRepresentation)

    # 외계행성 기준으로 좌표 변환
    relative_coords, new_distances = transform_coordinates(star_coords, exoplanet_coord)

    x_enu = relative_coords.x.value
    y_enu = relative_coords.y.value
    z_enu = relative_coords.z.value

    # 원래 절대 등급 계산
    df['abs_mag'] = df['Vmag'] - 5 * np.log10(df['distance']) + 5

    # 새로운 겉보기 등급 계산
    new_app_mag = df['abs_mag'] + 5 * np.log10(new_distances.value) - 5

    # 새로운 겉보기 등급을 데이터에 추가
    df['new_phot_g_mean_mag'] = new_app_mag

    # 정규화된 반지름 계산 (NaN 처리 포함)
    normalized_radius = calculate_normalized_radius(df['new_phot_g_mean_mag'].values)

    # DataFrame 업데이트
    df_processed = pd.DataFrame({
        'source_id': df.get('HIP'),
        'absolute_magnitude': df.get('abs_mag'),
        'apparent_magnitude': df.get('new_phot_g_mean_mag'),
        'radius': np.nan,
        'normalized_radius': normalized_radius,
        'temperature_celsius': np.nan,
        'distance_pc': new_distances.value,  # 외계행성 기준 거리 (parsec 단위)
        'x_enu': x_enu,
        'y_enu': y_enu,
        'z_enu': z_enu
    })

    # ENU 벡터의 노름 계산 (NaN 처리 포함)
    df_processed['enu_norm'] = np.sqrt(df_processed['x_enu']**2 + df_processed['y_enu']**2 + df_processed['z_enu']**2)

    # 노름이 0인 경우를 피하기 위해 작은 값을 더함 (필요시)
    epsilon = 1e-10
    df_processed['enu_norm'] = df_processed['enu_norm'].replace(0, epsilon)

    # 정규화된 ENU 좌표 계산 (단위 벡터)
    df_processed['x_normalized'] = df_processed['x_enu'] / df_processed['enu_norm']
    df_processed['y_normalized'] = df_processed['y_enu'] / df_processed['enu_norm']
    df_processed['z_normalized'] = df_processed['z_enu'] / df_processed['enu_norm']

    # 원래 ENU 좌표와 노름 열 삭제
    df_processed.drop(['x_enu', 'y_enu', 'z_enu', 'enu_norm'], axis=1, inplace=True)

    # 기타 필드에서 NaN 값 처리 (필요시)
    # 예: 절대 등급이 NaN인 경우 계산 불가, 같은 방식으로 처리할 수 있음
    df_processed['absolute_magnitude'] = df_processed['absolute_magnitude'].replace([np.inf, -np.inf], np.nan)
    df_processed['temperature_celsius'] = df_processed['temperature_celsius'].replace([np.inf, -np.inf], np.nan)


    # 필터링 및 파일 저장
    for mag in magnitude_limits:
        # 필터링: Vmag <= mag
        df_filtered = df_processed[df_processed['apparent_magnitude'] <= mag].copy()

        # 파일 이름 설정
        base_filename = f'HIP_{exoplanet_name}_mag{mag}'

        if 'json' in output_formats:
            output_json = f'{base_filename}.json'
            df_filtered.to_json(output_json, orient='records', lines=False, indent=4, default_handler=str)
            print(f"Saved {len(df_filtered)} stars with Vmag <= {mag} to '{output_json}'")

    print(f"Processing for {exoplanet_name} completed.\n")

# --- 메인 스크립트 ---

def main():
    # 겉보기 등급 기준 설정
    magnitude_limits = [4, 6]

    # 관측자 위치 설정 (위도, 경도)
    lat_deg = 37 + 23/60 + 41.01/3600  # 37.394725°
    lon_deg = 127 + 6/60 + 40.35/3600  # 127.1112083°
    observer_location = EarthLocation(lat=lat_deg * u.deg, lon=lon_deg * u.deg, height=0 * u.m)

    # 관측 시간 설정 (현재 시간)
    observation_time = Time.now()

    # 외계행성의 위치 정의
    exoplanets = {
        'K2-18': SkyCoord(
            ra='11h30m14.51774s',
            dec='+07d35m18.2553s',
            distance=38.10 * u.pc,
            frame='icrs'
        ),
        'Gliese_581': SkyCoord(
            ra='15h19m26.8250s',
            dec='-07d43m20.209s',
            distance=6.279 * u.pc,
            frame='icrs'
        )
    }

    # 각 외계행성에 대해 데이터 처리
    for exoplanet_name, exoplanet_coord in exoplanets.items():
        if exoplanet_name == 'K2-18':
            csv_filename = 'filtered_hipparcos_data_K2.csv'
        elif exoplanet_name == 'Gliese_581':
            csv_filename = 'filtered_hipparcos_data_Gliese.csv'
        else:
            print(f"Unknown exoplanet: {exoplanet_name}")
            continue

        # 데이터 처리 함수 호출
        process_exoplanet_data(
            csv_filename=csv_filename,
            exoplanet_coord=exoplanet_coord,
            exoplanet_name=exoplanet_name,
            magnitude_limits=magnitude_limits,
            observer_location=observer_location,
            observation_time=observation_time,
            output_formats=['json']  # JSON만 저장
        )

if __name__ == "__main__":
    main()


Saved 465 stars with Vmag <= 4 to 'HIP_K2-18_mag4.json'
Saved 4866 stars with Vmag <= 6 to 'HIP_K2-18_mag6.json'
Processing for K2-18 completed.

Saved 520 stars with Vmag <= 4 to 'HIP_Gliese_581_mag4.json'
Saved 4964 stars with Vmag <= 6 to 'HIP_Gliese_581_mag6.json'
Processing for Gliese_581 completed.



# 3. Merge the Two Data

In [ ]:
import json

# Load and read the two JSON files
with open('name_K2-18_mag8.json') as f1, open('name_HIP_Gliese_581_mag6.json') as f2:
    # Read and parse the JSON files (assuming each line is a JSON object)
    data1 = [json.loads(line) for line in f1]
    data2 = [json.loads(line) for line in f2]

# Merge the two lists
merged_data = data1 + data2

# Optionally, write the merged data to a new file
with open('K2-18_mag4.json', 'w') as mf:
    json.dump(merged_data, mf, indent=4)

print("JSON files have been merged successfully.")

JSON files have been merged successfully.


In [ ]:
# prompt: I want to organize JSON files in order with 'apparent_magnitude'

import json

def sort_json_by_magnitude(filename):
    """
    JSON 파일을 'apparent_magnitude'를 기준으로 정렬합니다.

    Args:
        filename: JSON 파일 이름

    Returns:
        정렬된 JSON 데이터 리스트
    """
    with open(filename) as f:
        data = json.load(f)

    # apparent_magnitude를 기준으로 정렬
    sorted_data = sorted(data, key=lambda x: x.get('apparent_magnitude', float('inf')))

    return sorted_data

# JSON 파일 이름 리스트
json_files = ['K2-18_mag4.json', 'K2-18_mag6.json', 'K2-18_mag8.json']  # 필요에 따라 파일 이름을 추가

# 각 파일을 정렬하고 저장
for filename in json_files:
    sorted_data = sort_json_by_magnitude(filename)

    # 정렬된 데이터를 다시 JSON 파일에 저장
    with open(filename, 'w') as f:
        json.dump(sorted_data, f, indent=4)

    print(f"File '{filename}' has been sorted by apparent_magnitude.")



File 'K2-18_mag4.json' has been sorted by apparent_magnitude.
File 'K2-18_mag6.json' has been sorted by apparent_magnitude.
File 'K2-18_mag8.json' has been sorted by apparent_magnitude.
